<a href="https://colab.research.google.com/github/giovannaily/Deep_Learn/blob/main/Fresh_and_rotten_fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade Federal de Pernambuco - UFPE \
Centro de Informática - CIn \
Ciência da computação - Pós graduação \
Deep Learn 2024.1 \
Student: Giovanna Ily Farias Ramalho

# Fresh and rotten fruits

In this notebook, I am training a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. My model require an output layer of 6 neurons.

Load Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/MyDrive/dataset/')

Mounted at /content/drive


Import pre-trainning model from image.net

In [ ]:
from tensorflow import keras

# Carregando o modelo VGG16 pré-treinado com pesos do ImageNet
# e sem as camadas totalmente conectadas no topo do modelo.
base_model = keras.applications.VGG16(
    weights='imagenet',  # Utiliza os pesos treinados no ImageNet
    input_shape=(224, 224, 3),  # Define a forma de entrada conforme requerido
    include_top=False)  # Não inclui as camadas superiores (fully-connected layers)

58889256/58889256 [==============================] - 4s 0us/step


Freeze the base model

In [ ]:
# Freeze base model
base_model.trainable = False

Add layers to model


In [ ]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Flatten, Dense

# 1. Criar os inputs com a forma correta
inputs = Input(shape=(224, 224, 3))

# 2. Passar inputs para o modelo base, definindo training como False para usar as camadas em modo de inferência
x = base_model(inputs, training=False)

# 3. Adicionar uma camada de pooling para reduzir a dimensionalidade ou uma camada Flatten
# Aqui vamos usar GlobalAveragePooling2D, mas você pode usar Flatten() se preferir
x = GlobalAveragePooling2D()(x)

# 4. Adicionar a camada densa final para classificação
# Suponha que temos 10 classes para este exemplo
outputs = Dense(10, activation='softmax')(x)

# 5. Combinar inputs e outputs para criar o modelo
model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                5130      
                                                                 
Total params: 14719818 (56.15 MB)
Trainable params: 5130 (20.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## Compile Model

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',  # Para labels em formato de inteiros
    metrics=['accuracy']
)

## Augment the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation para o conjunto de treinamento
datagen_train = ImageDataGenerator(
    rotation_range=40,       # Rotação de até 40 graus
    width_shift_range=0.2,   # Deslocamento horizontal até 20%
    height_shift_range=0.2,  # Deslocamento vertical até 20%
    shear_range=0.2,         # Cisalhamento até 20%
    zoom_range=0.2,          # Zoom de até 20%
    horizontal_flip=True,    # Permitir espelhamento horizontal
    fill_mode='nearest',     # Modo de preenchimento para novos pixels
    rescale=1./255           # Escala os valores de pixel para o intervalo [0,1]
)

# Apenas reescala para o conjunto de validação
datagen_valid = ImageDataGenerator(
    rescale=1./255           # Escala os valores de pixel para o intervalo [0,1]
)


## Load Dataset

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',  # Caminho para o diretório de treinamento
    target_size=(224, 224),                           # As imagens são redimensionadas para 224x224
    color_mode="rgb",                                 # Usar imagens em cores
    class_mode="categorical",                         # As labels serão one-hot encoded
    batch_size=32                                     # Processa 32 imagens por lote
)

# load and iterate "validation" dataset, now using a general directory that includes multiple folders
valid_it = datagen_valid.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',  # Caminho ajustado para o diretório que contém todas as subpastas
    target_size=(224, 224),                               # As imagens são redimensionadas para 224x224
    color_mode="rgb",                                     # Usar imagens em cores
    class_mode="categorical",                             # As labels serão one-hot encoded
    batch_size=32                                         # Processa 32 imagens por lote
)

Found 3998 images belonging to 6 classes.
Found 3998 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow import keras

# Suponho que você já tenha base_model configurado e não treinável
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(6, activation='softmax')(x)  # Corrigindo o número de neurônios para 6
model = keras.Model(inputs=inputs, outputs=outputs)

# Recompile o modelo após ajustar a camada de saída
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
# Verificação de class_mode e ajustes do ImageDataGenerator
datagen_train = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)
train_it = datagen_train.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32
)

datagen_valid = ImageDataGenerator(rescale=1./255)
valid_it = datagen_valid.flow_from_directory(
    '/content/drive/MyDrive/dataset/train',
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32
)


Found 3998 images belonging to 6 classes.
Found 3998 images belonging to 6 classes.


## Train the Model

In [ ]:
model.fit(
    train_it,  # Gerador de dados de treinamento
    validation_data=valid_it,  # Gerador de dados de validação
    steps_per_epoch=train_it.samples // train_it.batch_size,  # Usando divisão inteira para garantir um número inteiro
    validation_steps=valid_it.samples // valid_it.batch_size,  # Usando divisão inteira para garantir um número inteiro
    epochs=10  # Definindo um número de épocas para treinamento
)


Epoch 1/10
124/124 [==============================] - 1713s 14s/step - loss: 0.8957 - accuracy: 0.7254 - val_loss: 0.6132 - val_accuracy: 0.8019
Epoch 2/10
124/124 [==============================] - 122s 982ms/step - loss: 0.5863 - accuracy: 0.8174 - val_loss: 0.4414 - val_accuracy: 0.8198
Epoch 3/10
124/124 [==============================] - 113s 916ms/step - loss: 0.4572 - accuracy: 0.8704 - val_loss: 0.3463 - val_accuracy: 0.8642
Epoch 4/10
124/124 [==============================] - 114s 920ms/step - loss: 0.3794 - accuracy: 0.9034 - val_loss: 0.2931 - val_accuracy: 0.8851
Epoch 5/10
124/124 [==============================] - 121s 978ms/step - loss: 0.3320 - accuracy: 0.9181 - val_loss: 0.2437 - val_accuracy: 0.9332
Epoch 6/10
124/124 [==============================] - 121s 977ms/step - loss: 0.2966 - accuracy: 0.9266 - val_loss: 0.2328 - val_accuracy: 0.9050
Epoch 7/10
124/124 [==============================] - 122s 988ms/step - loss: 0.2668 - accuracy: 0.9355 - val_loss: 0.1968 - 

## Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

124/124 [==============================] - 34s 271ms/step - loss: 0.1712 - accuracy: 0.9370


[0.1711609959602356, 0.9369685053825378]

#Conclusão

Eu avaliei meu modelo nos dados de validação e observei resultados sólidos, o que me dá uma boa indicação de como o modelo provavelmente se comportará em dados não vistos, assumindo que os dados de validação são representativos do problema geral que estou tentando resolver. Aqui estão os detalhes e considerações baseados nos resultados fornecidos:

1. **Perda (Loss)**: O valor da perda que eu observei é 0.1712, que é relativamente baixo. Isso indica que, em média, meu modelo tem um bom desempenho em minimizar a função de perda definida durante o treinamento. A função de perda quantifica o quanto as previsões do modelo desviam dos valores reais, portanto, quanto menor o valor, melhor.

2. **Acurácia**: A acurácia de 93.70% é bastante alta, o que sugere que meu modelo está classificando corretamente uma grande proporção das amostras de validação. Este é um bom indicativo de desempenho, especialmente em tarefas de classificação onde a acurácia é uma métrica relevante e direta para avaliar a performance.

3. **Análise de Overfitting e Underfitting**:
   - **Overfitting**: Se a acurácia do treinamento for significativamente mais alta do que a acurácia de validação, isso pode indicar overfitting. Meu modelo seria então muito bom em prever dados de treinamento, mas não tão bom em generalizar para novos dados.
   - **Underfitting**: Se ambas as acurácias, de treinamento e de validação, forem baixas, ou se a acurácia de validação for inesperadamente mais alta do que a de treinamento, isso pode indicar underfitting. Significaria que meu modelo é demasiado simples para capturar padrões complexos nos dados.

4. **Consistência com Treinamento**: Comparando com os resultados anteriores de treinamento que compartilhei, onde a acurácia de validação alcançou até 95.11% em uma das épocas, a acurácia de 93.70% está um pouco abaixo, mas ainda assim é consistente. Isso pode ser devido a variações naturais nos diferentes subconjuntos de dados ou a pequenas diferenças na maneira como os dados de validação são processados ou selecionados.

5. **Tempo por Etapa**: Cada etapa levou em média 271ms, o que é relativamente rápido e mostra que meu modelo é computacionalmente eficiente com os recursos atuais. Isso é importante se o modelo precisar ser implementado em um ambiente de produção onde o tempo de resposta é crítico.

**Conclusão**:
Parece que meu modelo está bem ajustado com uma boa performance na validação. No entanto, sempre há espaço para melhorias, especialmente se eu notar alguma tendência de overfitting ou se precisar de uma acurácia ainda maior. Eu poderia explorar ajustes adicionais nos hiperparâmetros, aumentar o conjunto de dados de treinamento, ou incorporar técnicas de regularização e augmentação de dados, conforme mencionado anteriormente.